In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

import numpy as np
import pandas as pd

import spacy
from spacy.training import offsets_to_biluo_tags
import en_core_web_sm

nlp = en_core_web_sm.load()

from tqdm import trange
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaConfig, RobertaForTokenClassification, AdamW

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Example of RoBERTaForTokenClassification initialization
model = RobertaForTokenClassification.from_pretrained('roberta-base')

# Rest of your code here...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import math
print('TF version',tf.__version__)

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


TF version 2.15.0


In [ ]:
!pip install --no-deps seqeval[gpu]
from seqeval.metrics import classification_report, accuracy_score, f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=077454591fc8b6c52829d5ef6520884bb073ec3847a9697eacdf89627851bd9b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
df = pd.read_json('/content/drive/MyDrive/Colab_Notebooks/Entity Recognition in Resumes.json', lines=True)
df.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [ ]:
# Adding '\n' to the default spacy tokenizer

prefixes = ('\\n',) + tuple(nlp.Defaults.prefixes)

prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [ ]:
entity_dict = {
    'Name': 'NAME',
    'College Name': 'CLG',
    'Degree': 'DEG',
    'Graduation Year': 'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at': 'COMPANY',
    'Designation': 'DESIG',
    'Skills': 'SKILLS',
    'Location': 'LOC',
    'Email Address': 'EMAIL'
}

In [ ]:
# Checking for unique values present in 'extras' column
df['extras'].unique()

array([nan])

In [ ]:
# Since, 'extras' column contains no information we can drop the column
df = df.drop(['extras'], axis=1)
df.head()

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [ ]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [ ]:
def get_entities(df):

    entities = []

    for i in range(len(df)):
        entity = []

        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass

        entity = mergeIntervals(entity)
        entities.append(entity)

    return entities

In [ ]:
# Adding a new column 'entities'
df['entities'] = get_entities(df)
df.head()

,content,annotation,entities
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, NAME), (13, 46, DESIG), (49, 58, COMP..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, NAME), (62, 68, LOC), (104, 148, EMAI..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, NAME), (22, 31, LOC), (65, 117, EMAIL..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, NAME), (13, 51, DESIG), (54, 60, COMP..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, NAME), (14, 22, DESIG), (24, 41, COMP..."


In [ ]:
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]

        doc = nlp(text)

        tag = offsets_to_biluo_tags(doc, entities)
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text == '.' and tmp[1][i] == 'O':
                loc.append(i)
        loc.append(len(doc))

        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos

        for d in data:
            tag = ['O' if t == '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)

    return sentences, tags

In [ ]:
sentences, tags = get_train_data(df)
len(sentences), len(tags)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[(0, 14, 'NAME'), (62, 68, 'LOC'), (104, 148, 'EMA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akhil Yadav Polemaina
Hyderabad, Telangana - Email..." with entities "[(0, 21, 'NAME'), (22, 31, 'LOC'), (65, 117, 'EMAI...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

(780, 780)

In [ ]:
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])
tag_vals

{'B-CLG',
 'B-COMPANY',
 'B-DEG',
 'B-DESIG',
 'B-EMAIL',
 'B-GRADYEAR',
 'B-LOC',
 'B-NAME',
 'B-SKILLS',
 'B-YOE',
 'I-CLG',
 'I-COMPANY',
 'I-DEG',
 'I-DESIG',
 'I-EMAIL',
 'I-GRADYEAR',
 'I-LOC',
 'I-NAME',
 'I-SKILLS',
 'I-YOE',
 'L-CLG',
 'L-COMPANY',
 'L-DEG',
 'L-DESIG',
 'L-EMAIL',
 'L-GRADYEAR',
 'L-LOC',
 'L-NAME',
 'L-SKILLS',
 'L-YOE',
 'O',
 'U-CLG',
 'U-COMPANY',
 'U-DEG',
 'U-DESIG',
 'U-EMAIL',
 'U-GRADYEAR',
 'U-LOC',
 'U-SKILLS',
 'U-YOE',
 'X',
 '[CLS]',
 '[SEP]'}

In [ ]:
tag2idx = {t: i for i, t in enumerate(tag_vals)}
tag2idx

{'X': 0,
 'B-DESIG': 1,
 'I-EMAIL': 2,
 'U-GRADYEAR': 3,
 'L-DESIG': 4,
 'I-YOE': 5,
 'L-NAME': 6,
 'U-DESIG': 7,
 'L-COMPANY': 8,
 'L-SKILLS': 9,
 'U-LOC': 10,
 'B-GRADYEAR': 11,
 'L-LOC': 12,
 'I-LOC': 13,
 'U-EMAIL': 14,
 'O': 15,
 'U-COMPANY': 16,
 'I-GRADYEAR': 17,
 'U-DEG': 18,
 'B-COMPANY': 19,
 'B-YOE': 20,
 '[CLS]': 21,
 '[SEP]': 22,
 'U-YOE': 23,
 'B-CLG': 24,
 'L-YOE': 25,
 'U-SKILLS': 26,
 'I-SKILLS': 27,
 'L-DEG': 28,
 'B-EMAIL': 29,
 'B-LOC': 30,
 'I-CLG': 31,
 'L-EMAIL': 32,
 'B-SKILLS': 33,
 'L-GRADYEAR': 34,
 'B-DEG': 35,
 'B-NAME': 36,
 'U-CLG': 37,
 'I-DEG': 38,
 'I-NAME': 39,
 'I-COMPANY': 40,
 'L-CLG': 41,
 'I-DESIG': 42}

In [ ]:
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}
idx2tag

{0: 'X',
 1: 'B-DESIG',
 2: 'I-EMAIL',
 3: 'U-GRADYEAR',
 4: 'L-DESIG',
 5: 'I-YOE',
 6: 'L-NAME',
 7: 'U-DESIG',
 8: 'L-COMPANY',
 9: 'L-SKILLS',
 10: 'U-LOC',
 11: 'B-GRADYEAR',
 12: 'L-LOC',
 13: 'I-LOC',
 14: 'U-EMAIL',
 15: 'O',
 16: 'U-COMPANY',
 17: 'I-GRADYEAR',
 18: 'U-DEG',
 19: 'B-COMPANY',
 20: 'B-YOE',
 21: '[CLS]',
 22: '[SEP]',
 23: 'U-YOE',
 24: 'B-CLG',
 25: 'L-YOE',
 26: 'U-SKILLS',
 27: 'I-SKILLS',
 28: 'L-DEG',
 29: 'B-EMAIL',
 30: 'B-LOC',
 31: 'I-CLG',
 32: 'L-EMAIL',
 33: 'B-SKILLS',
 34: 'L-GRADYEAR',
 35: 'B-DEG',
 36: 'B-NAME',
 37: 'U-CLG',
 38: 'I-DEG',
 39: 'I-NAME',
 40: 'I-COMPANY',
 41: 'L-CLG',
 42: 'I-DESIG'}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/tokenizer.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaF

In [ ]:
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):

        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']

        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')

        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')

        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)

    return tokenized_texts, word_piece_labels

In [ ]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)

In [ ]:
print(tokenized_texts[0])
print(word_piece_labels[0])

['[CLS]', 'Ab', 'his', 'he', 'k', 'J', 'ha', 'Ċ', 'Application', 'Development', 'Ass', 'ociate', '-', 'Acc', 'enture', 'Ċ', 'Ċ', 'B', 'eng', 'al', 'uru', ',', 'K', 'arn', 'ataka', '-', 'Email', 'me', 'on', 'Indeed', ':', 'ind', 'eed', '.', 'com', '/', 'r', '/', 'Ab', 'his', 'he', 'k', '-', 'J', 'ha', '/', '10', 'e', '7', 'a', '8', 'cb', '7', '32', 'bc', '43', 'a', 'Ċ', 'Ċ', 'âĢ¢', 'To', 'work', 'for', 'an', 'organ', 'ization', 'which', 'prov', 'ides', 'me', 'the', 'opp', 'ortun', 'ity', 'to', 'improve', 'my', 'sk', 'ills', 'Ċ', 'and', 'knowledge', 'for', 'my', 'individual', 'and', 'company', "'s", 'growth', 'in', 'best', 'p', 'ossible', 'ways', '[SEP]']
['[CLS]', 'B-NAME', 'X', 'X', 'X', 'L-NAME', 'X', 'O', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'X', 'O', 'U-COMPANY', 'X', 'O', 'O', 'U-LOC', 'X', 'X', 'X', 'O', 'O', 'X', 'X', 'O', 'O', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 

In [ ]:
MAX_LEN = 512
bs = 4

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))
print(input_ids[0])


512
[    3 13112 12724   700   330   863  1999 50118 46345 45297 26039 43097
    12 36984 20537 50118 50118   387  3314   337  5515     6   530  4422
 15078    12 46084  1794   261 40566    35  2028 10247     4   175    73
   338    73 13112 12724   700   330    12   863  1999    73   698   242
   406   102   398 38133   406  2881 23219  3897   102 50118 50118 21438
  3972  6014  1990   260 17247  1938  5488 13138  4376  1794   627 10223
 21108  1571   560 37362  4783  7771  5622 50118   463 44228  1990  4783
 42333   463 24233    18 14596   179  7885   642 31497  5924     3     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels], maxlen=MAX_LEN, value=tag2idx["O"],
                     padding="post", dtype="long", truncating="post")
print(len(tags[0]))
print(tags[0])

512
[21 36  0  0  0  6  0 15  1 42  4  0 15 16  0 15 15 10  0  0  0 15 15  0
  0 15 15 15 15 29  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0 32 15 15 15 15 15 15 15  0 15 15  0 15 15 15
  0  0 15 15 15 15  0 15 15 15 15 15 15 15 15 15 15 15 15 15  0 15 22 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 1

In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=2020,
                                                                                 test_size=0.3)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
model = RobertaForTokenClassification.from_pretrained('roberta-base',  num_labels=len(tag2idx))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",

In [ ]:
model.cuda();

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
epochs=10
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        # extract loss from outputs
        loss = outputs.loss
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:   0%|          | 0/10 [00:48<?, ?it/s]


NameError: name 'max_grad_norm' is not defined

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

model.eval()

y_true = []
y_pred = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        logits = model(input_ids, token_type_ids=None, attention_mask=input_mask,)

    logits = logits.logits.cpu().numpy()

    logits = [list(p) for p in np.argmax(logits, axis=2)]

    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()

    for i,mask in enumerate(input_mask):
        temp_1 = [] # Real one
        temp_2 = [] # Predict one

        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if idx2tag[label_ids[i][j]] != "X" and idx2tag[label_ids[i][j]] != "[CLS]" and idx2tag[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(idx2tag[label_ids[i][j]])
                    temp_2.append(idx2tag[logits[i][j]])
            else:
                break

        y_true.append(temp_1)
        y_pred.append(temp_2)

# Convert sequence of sequences labels to binary array format
mlb = MultiLabelBinarizer()
y_true_binary = mlb.fit_transform(y_true)
y_pred_binary = mlb.transform(y_pred)

# Compute metrics using binary array format
print("f1 score: %f" % (f1_score(y_true_binary, y_pred_binary, average='weighted')))
print("Accuracy score: %f" % (accuracy_score(y_true_binary, y_pred_binary)))

print(classification_report(y_true_binary, y_pred_binary, digits=4))


f1 score: 0.768393
Accuracy score: 0.192308
              precision    recall  f1-score   support

           0     0.7556    0.6800    0.7158        50
           1     0.5532    0.5652    0.5591        46
           2     0.8000    0.8980    0.8462        49
           3     0.7653    0.8333    0.7979        90
           4     0.4500    0.7826    0.5714        23
           5     0.1667    0.3333    0.2222         3
           6     0.6667    0.1818    0.2857        11
           7     0.9825    0.9492    0.9655        59
           8     0.9455    0.8254    0.8814        63
           9     0.5556    1.0000    0.7143         5
          10     0.6579    0.5814    0.6173        43
          11     0.2632    0.2778    0.2703        18
          12     0.7778    0.9130    0.8400        46
          13     0.5781    0.7115    0.6379        52
          14     0.7059    1.0000    0.8276        12
          15     1.0000    1.0000    1.0000         1
          16     0.0000    0.0000    

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['X'] will be ignored
  warnings.warn(
